In [4]:
# 환경 변수 로드
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="./env/.env")

api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit = 120,
    memory_key = "chat_history", # 프롬프트의 {chat_history}와 연결
    return_messages = True # 메시지 객체(리스트) 형태로 결과를 반환하도록 설정
)

# MessagesPlaceholder: 메모리에서 가져온 메시지 리스트가 들어갈 위치 지정
# 'chat_history' 변수명을 메모리의 momory_key와 일치시켜서 주입
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name = "chat_history"),
    ("human", "{question}"),
])

# 메모리 로드 함수:
# LCEL 파이프라인은 자동으로 메모리를 읽지 못함
# 'chat_history'라는 이름으로 과거 대화 리스트만 꺼내서 반환
def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]


# RunnablePassthrough.assign: 
# - 사용자가 보낸 {"question": "..."} 데이터를 그대로 받아서 유지함
# - 동시에 load_memory를 실행해서 'chat_history' 라는 이름으로 결과값을 옆에 붙임
# - 최종적으로 {"question": "...", "chat_hisyory": [...]} 형태를 만들어 prompt에 전달
chain = RunnablePassthrough.assign(chat_history = load_memory) | prompt | llm

def invoke_chain(question):
   # RunnablePassthrough.assign 덕분에 invoke에서 메모리 신경쓰지 않고 질문만 해도 됨
   result = chain.invoke({"question": question})
   
   # 메모리 저장은 수동으로 해야함 (LCEL)
   memory.save_context(
       {"input": question}, 
       {"output": result.content},
    )
   
   print(result)


In [6]:
invoke_chain("My name is Yujin")

content='Hello Yujin! How can I assist you today?'


In [7]:
invoke_chain("What is my name?")

content='Your name is Yujin.'
